In [ ]:
import sys
from pathlib import Path
import pandas as pd
from cogwheel import gw_plotting, gw_utils, utils
sys.path.insert(0, str(Path.cwd().parent))
from dot_pe import sample_banks, config

In [ ]:
bank_dir = Path("test_bank")
sample_banks.main(
    bank_size=2**12,
    q_min=1 / 6,
    m_min=50,
    m_max=100,
    inc_faceon_factor=1.0,
    f_ref=50.0,
    fbin=config.DEFAULT_FBIN,
    n_pool=4,
    blocksize=1024,
    approximant="IMRPhenomXODE",
    bank_dir=bank_dir,
)

In [ ]:
samples = pd.read_feather(bank_dir / "intrinsic_sample_bank.feather")
samples["mchirp"] = gw_utils.m1m2_to_mchirp(samples["m1"], samples["m2"])
samples["chieff"] = gw_utils.chieff(*samples[["m1", "m2", "s1z", "s2z"]].values.T)

weighted_samples = samples.copy()
weighted_samples["weights"] = utils.exp_normalize(
    weighted_samples["log_prior_weights"].values
)
samples["weights"] = 1.0
gw_plotting.MultiCornerPlot(
    [samples, weighted_samples],
    smooth=1,
    params=[c for c in samples.columns if "weights" not in c],
    labels=["Unweighted", "Weighted"],
).plot()